In [1]:
!pip install torchmetrics

     |████████████████████████████████| 332 kB 13.0 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import nibabel as nib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
import math
from functools import partial
from torchmetrics import Accuracy


In [4]:
case = os.getcwd() + '/drive/MyDrive/masks'
print(case)

/content/drive/MyDrive/masks


In [5]:
for en, i in enumerate(os.walk(case)):
            if en>0:
                gd = glob.glob(i[0] + '/*GD3D.*')
                t2 = glob.glob(i[0] + '/*T2.*')
                flair = glob.glob(i[0] + '/*FLAIR.*')

                edema_mask = glob.glob(i[0] + '/*EDEMA_MA*')
                folder = os.path.basename(i[0])
                print(folder)
                
                print(t2)
                print(edema_mask)

                print('case number ' + folder + ' max T1GD3D:',nib.load(gd[0]).get_fdata().max())
                print('case number ' + folder + ' max flair:',nib.load(flair[0]).get_fdata().max())

                print('case number ' + folder + ' edmea:',np.unique(nib.load(edema_mask[0]).get_fdata()))
                
                print('case number ' + folder + ' max T2:',nib.load(t2[0]).get_fdata().max())

                print('-----------------------------------')

108
['/content/drive/MyDrive/masks/108/108T2.nii.gz']
['/content/drive/MyDrive/masks/108/108EDEMA_MASK.nii.gz']
case number 108 max T1GD3D: 636.1326904296875
case number 108 max flair: 780.05078125
case number 108 edmea: [0. 1.]
case number 108 max T2: 540.1934814453125
-----------------------------------
102
['/content/drive/MyDrive/masks/102/102T2.nii.gz']
['/content/drive/MyDrive/masks/102/102EDEMA_MASK.nii.gz']
case number 102 max T1GD3D: 1043.44287109375
case number 102 max flair: 428.68316650390625
case number 102 edmea: [0. 1.]
case number 102 max T2: 537.252685546875
-----------------------------------
106
['/content/drive/MyDrive/masks/106/106T2.nii.gz']
['/content/drive/MyDrive/masks/106/106EDEMA_MASK.nii.gz']
case number 106 max T1GD3D: 927.1036376953125
case number 106 max flair: 299.4729309082031
case number 106 edmea: [0. 1.]
case number 106 max T2: 850.5272827148438
-----------------------------------
101
['/content/drive/MyDrive/masks/101/101T2.nii.gz']
['/content/drive

##  Dataset

In [6]:
dataframe = pd.read_excel(case + '/DATALABLLLLL.xlsx')

In [7]:
dataframe = pd.read_csv(case +'/ty.csv')

In [8]:
le = preprocessing.LabelEncoder()
prim_label = le.fit_transform(dataframe['Primary metastasis site'])

In [9]:
prim_label

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 2, 0, 2, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 2, 0, 1, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0])

In [10]:
dataframe.to_csv('ty.csv')

In [11]:
dataframe

,Unnamed: 0,Name of pateient's,Number of file,Primary metastasis site,Gender,Edema Volume,Tumor Volume,WOHLEBRAIN_VOLUME,NUMBER OF TUMOR,Age
0,0,4,15956,breast,female,82387000000,6481000000,1975515000000,1,83
1,1,6,7865,breast,female,11638000000,2789000000,1923697000000,3,62
2,2,12,5537,breast,female,1922000000,2683000000,1974133000000,1,33
3,3,16,6682,breast,female,4609000000,2449000000,2014619000000,1,44
4,4,18,13914,breast,female,53359000000,15793000000,1976792000000,4,50
5,5,20,2432,breast,female,14376000000,1290000000,1984177000000,2,60
6,6,24,15481,lung,male,1646000000,545000000,1965162000000,1,61
7,7,25,17131,lung,male,59284000000,11034000000,1951091000000,2,61
8,8,26,10900,lung,female,4408000000,1488000000,2022031000000,1,50
9,9,28,9949,lung,female,9534000000,3084000000,1995644000000,3,33


In [12]:
def normalization(image):
    smooth = 1e-9
    mean = np.mean(image, 1, keepdims=True)
    std = np.std(image,1,keepdims=True)
    return (image - mean + smooth)/(std + smooth)

In [13]:
class dataset(torch.utils.data.Dataset):
    def __init__(self, flag = 1, mode = 'train'):
        super().__init__()
        self.gd = []
        self.flag = flag
        self.t2 = []
        self.flair = []
        self.edema_mask = []
        self.gd_mask = []
        for en, i in enumerate(os.walk(case)):
            if en>0:
                folder = os.path.basename(i[0])

                
                gd = glob.glob(i[0] + '/*GD3D.*')
                t2 = glob.glob(i[0] + '/*T2.*')
                flair = glob.glob(i[0] + '/*FLAIR.*')
                edema_mask = glob.glob(i[0] + '/*EDEMA_MA*')
                gd_mask = glob.glob(i[0] + '/*GD3D_M*')
                
                self.gd.append(gd[0])
                self.gd_mask.append(gd_mask[0])
                self.t2.append(t2[0])
                self.flair.append(flair[0])
                self.edema_mask.append(edema_mask[0])
                
                
                self.flair.sort(key = lambda x : int(x.split('/')[-2]))
                self.edema_mask.sort(key = lambda x : int(x.split('/')[-2]))
                self.t2.sort(key = lambda x : int(x.split('/')[-2]))
                self.gd.sort(key = lambda x : int(x.split('/')[-2]))
                self.gd_mask.sort(key = lambda x : int(x.split('/')[-2]))
                
        tr = int(0.7 * len(self.gd))
        test = int(0.2 * len(self.gd))
        if mode == 'train':
            self.gd_g=self.gd[:tr]
            self.gd_mask_g=self.gd_mask[:tr]
            self.t2_g=self.t2[:tr]
            self.flair_g=self.flair[:tr]
            self.edema_mask_g=self.edema_mask[:tr]
        elif mode == 'test':
            self.gd_g=self.gd[tr:tr +test]
            self.gd_mask_g=self.gd_mask[tr:tr +test]
            self.t2_g=self.t2[tr:tr +test]
            self.flair_g=self.flair[tr:tr +test]
            self.edema_mask_g=self.edema_mask[tr:tr +test]
        elif mode == 'valid':
            self.gd_g=self.gd[tr +test:]
            self.gd_mask_g=self.gd_mask[tr +test:]
            self.t2_g=self.t2[tr +test:]
            self.flair_g=self.flair[tr +test:]
            self.edema_mask_g=self.edema_mask[tr +test:]

    def __len__(self):
        
        return len( self.gd_g)
        
        
    def __getitem__(self, ind):
        
        case = self.gd_g[ind].split('/')[-2]    
        t1 = nib.load(self.gd_g[ind]).get_fdata()
        t1_mask = nib.load(self.gd_mask_g[ind]).get_fdata()
        t1_normal = normalization(t1)
        t1_inten_normal = t1_normal * t1_mask
        
        t2 = nib.load(self.t2_g[ind]).get_fdata()
        edema_mask = nib.load(self.edema_mask_g[ind]).get_fdata()
        t2_normal = normalization(t2)
        t2_inten_normal = t2_normal * edema_mask
        
        flair = nib.load(self.flair_g[ind]).get_fdata()
        flair_normal = normalization(flair)
        flair_inten_normal = flair_normal * edema_mask
        both = np.where(edema_mask==1., flair_inten_normal, t1_inten_normal)
        if self.flag==1:
            cat = np.transpose(t1_inten_normal, (1,0,2))
            cat = np.expand_dims(cat,0)
            cat = np.float32(cat)
            return {'t1' : cat,'label': prim_label[ind], 'case': case}
        elif self.flag ==2:
            cat = np.transpose(both, (1,0,2))
            cat = np.expand_dims(cat,0)
            cat = np.float32(cat)
            return {'t1_flair' : cat,'label': prim_label[ind], 'case' : case}
         
        elif self.flag ==3:
            cat = np.concatenate([t1_inten_normal, flair_inten_normal, t2_inten_normal], 1)
            cat = np.transpose(cat, (1,0,2))
            cat = np.expand_dims(cat,0)
            cat = np.float32(cat)
            return {'t1_flair_t2' : cat,'label': prim_label[ind]}
         
        

In [14]:
d_train = dataset(flag = 1,mode = 'train') 
d_test = dataset(flag = 1,mode = 'test') 
d_valid = dataset(flag = 1,mode = 'valid') 

In [15]:
dataloader_train = torch.utils.data.DataLoader(d_train, batch_size = 1, shuffle = True)
dataloader_test = torch.utils.data.DataLoader(d_test, batch_size = 1, shuffle = False)
dataloader_valid = torch.utils.data.DataLoader(d_valid, batch_size = 1, shuffle = False)

# resnet3d

In [16]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1x1 = nn.Conv3d(in_planes,planes,1,bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv3x3 = nn.Conv3d(planes, planes, 3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv1x1_1 = nn.Conv3d(planes, planes * self.expansion, 1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1x1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv3x3(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv1x1_1(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 block_inplanes,
                 n_input_channels=1,
                 conv1_t_size=7,
                 conv1_t_stride=1,
                 no_max_pool=False,
                 shortcut_type='B',
                 widen_factor=1.0,
                 n_classes=3):
        super().__init__()

        block_inplanes = [int(x * widen_factor) for x in block_inplanes]

        self.in_planes = block_inplanes[0]
        self.no_max_pool = no_max_pool

        self.conv1 = nn.Conv3d(n_input_channels,
                               self.in_planes,
                               kernel_size=(conv1_t_size, 7, 7),
                               stride=(conv1_t_stride, 2, 2),
                               padding=(conv1_t_size // 2, 3, 3),
                               bias=False)
        self.bn1 = nn.BatchNorm3d(self.in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, block_inplanes[0], layers[0],
                                       shortcut_type)
        self.layer2 = self._make_layer(block,
                                       block_inplanes[1],
                                       layers[1],
                                       shortcut_type,
                                       stride=2)
        self.layer3 = self._make_layer(block,
                                       block_inplanes[2],
                                       layers[2],
                                       shortcut_type,
                                       stride=2)
        self.layer4 = self._make_layer(block,
                                       block_inplanes[3],
                                       layers[3],
                                       shortcut_type,
                                       stride=2)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(block_inplanes[3] * block.expansion, n_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _downsample_basic_block(self, x, planes, stride):
        out = F.avg_pool3d(x, kernel_size=1, stride=stride)
        zero_pads = torch.zeros(out.size(0), planes - out.size(1), out.size(2),
                                out.size(3), out.size(4))
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.cuda()

        out = torch.cat([out.data, zero_pads], dim=1)

        return out

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(self._downsample_basic_block,
                                     planes=planes * block.expansion,
                                     stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(self.in_planes, planes * block.expansion, 1, stride),
                    nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(
            block(in_planes=self.in_planes,
                  planes=planes,
                  stride=stride,
                  downsample=downsample))
        self.in_planes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if not self.no_max_pool:
            x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [17]:
model = ResNet(Bottleneck, [3, 4, 6, 3], [64, 128, 256, 512], n_classes=3)

# Model_resnet2d

In [18]:
class resnet_block(nn.Module):
    expansion = 4
    
    def __init__(self, input_activation, intermediate, stride = 1):
        super().__init__()
        output = intermediate * self.__class__.expansion

        self.conv1x1_1 = nn.Conv2d(input_activation, intermediate, 1)
        self.BN1 = nn.BatchNorm2d(intermediate)
        
        self.conv3x3 =  nn.Conv2d(intermediate, intermediate, 3, stride=stride, padding=1)
        self.BN2 = nn.BatchNorm2d(intermediate)

        self.conv1x1_2 =  nn.Conv2d(intermediate, output, 1)
        self.BN3 = nn.BatchNorm2d(output)
        
        self.down = None
        if ( input_activation != output ) or ( stride > 1 ) :
            
            self.down = nn.Sequential(
                                      nn.Conv2d(input_activation, output, 1, stride=stride),
                                      nn.BatchNorm2d(output)
                                      )
        
    def forward(self , inp):
        inp1 = inp
        c = F.relu(self.BN1(self.conv1x1_1(inp)))
        c = F.relu(self.BN2(self.conv3x3(c)))
        c = self.BN3(self.conv1x1_2(c))
        if self.down is not None:
            inp1 = self.down(inp)
        out = F.relu(c + inp1) 
        
        return out    
    

In [ ]:
class resnet_2d(nn.Module):
  

    def __init__(self, layers, width=64, num_class):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # residual layers
        self._inplanes = width  # this is a *mutable* variable used during construction
        self.layer1 = self._make_layer(width, layers[0])
        self.layer2 = self._make_layer(width * 2, layers[1], stride=2)
        self.layer3 = self._make_layer(width * 4, layers[2], stride=2)
        self.layer4 = self._make_layer(width * 8, layers[3], stride=2)

        embed_dim = width * 32  # the ResNet feature dimension
        self.linear = nn.Linear(embed_dim,num_class)
        self.out = nn.Linear()
    def _make_layer(self, planes, blocks, stride=1):
        layers = [resnet_block(self._inplanes, planes, stride)]

        self._inplanes = planes * Bottleneck.expansion
        for _ in range(1, blocks):
            layers.append(resnet_block(self._inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        batch = x.shape[0]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x_view = x.view(batch, -1)
        x = self.linear(x_view)
        return x


SyntaxError: ignored

In [ ]:
model2d = resnet_2d([3,4,6,3])

NameError: ignored

# Train

In [19]:
device = torch.device("cuda:0" if  torch.cuda.is_available() else "cpu")
accuracy = Accuracy()

In [20]:
device

device(type='cuda', index=0)

In [27]:

def train_step(model, dataload_train , dataload_valid ,epochs = 70,
               lr = 0.001):

    epochs = epochs
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    hist_loss_per_epoch = []
    hist_acc_per_epoch = []
    hist_acc_tr_per_epoch = []

    for epoch in range(epochs):
        hist_loss_per_batch = []
        for batch, data in enumerate(dataload_train, 1):
            image = data['t1'].to(device)
            segment = data['label'].to(device)
            segment_hat = model(image)
            loss = F.cross_entropy(segment_hat, segment) 
            hist_loss_per_batch.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
        mean_loss = sum(hist_loss_per_batch)/batch
        hist_loss_per_epoch.append(mean_loss)
        print(f'Loss in epoch{epoch} is :', mean_loss)
        
        hist_ac_tr_per_batch = []
        for batch, data in enumerate(dataload_train, 1):
            image = data['t1'].to(device)
            segment = data['label'].to(device)
            with torch.set_grad_enabled(False):
                segment_hat = model(image)
                ac = accuracy(segment_hat.cpu(), segment.cpu())
            hist_ac_tr_per_batch.append(ac)
        
        mean_ac_tr = sum(hist_ac_tr_per_batch)/batch
        hist_acc_tr_per_epoch.append(mean_ac_tr)
        print(f'acc train in epoch{epoch} is :', mean_ac_tr)

        
        hist_ac_per_batch = []
        for batch, data in enumerate(dataload_valid, 1):
            image = data['t1'].to(device)
            segment = data['label'].to(device)
            with torch.set_grad_enabled(False):
                segment_hat = model(image)
                ac = accuracy(segment_hat.cpu(), segment.cpu())
            hist_ac_per_batch.append(ac)
        
        mean_ac = sum(hist_ac_per_batch)/batch
        hist_acc_per_epoch.append(mean_ac)
        print(f'acc valid in epoch{epoch} is :', mean_ac)

    
      
        
    torch.save({ "epoch" : epoch, 
                         "model_state" : model.state_dict(),
                         "optimizer" : optimizer.state_dict(),
                         "hist_loss" : hist_loss_per_epoch,
                        "hist_val" :  hist_acc_per_epoch

                       },

                       f"model_epoch_{epoch}.pt")

        
    return hist_loss_per_epoch, hist_acc_per_epoch



                

In [ ]:
his = train_step(model, dataloader_train, dataloader_valid)

Loss in epoch0 is : 1.6346846604874978


KeyError: ignored

In [ ]:
ma = torch.load('/content/drive/MyDrive/masks/model_flag2_100.pt')

In [ ]:
hist_val = ma['hist_val']
hist_loss = ma['hist_loss']
model.load_state_dict(ma['model_state'])

<All keys matched successfully>

In [ ]:
hist_loss

[1.6166587988535563,
 1.2716993123292923,
 1.2022380198041598,
 1.0400173167387645,
 0.980600060025851,
 1.0389837632576624,
 1.0905448953310648,
 0.9533298159639041,
 0.9632542292277019,
 1.0532008518775304,
 1.3045772473017374,
 1.0198955997824668,
 0.8883888761202494,
 0.8700950831174851,
 0.9450457776586215,
 0.7930878539880116,
 0.8246473530928294,
 0.751122119029363,
 0.965070107827584,
 0.7983172694842021,
 0.7165512065092723,
 0.7870256179322799,
 0.7117170721292496,
 0.806572666267554,
 0.6083797191580137,
 0.4795561968038479,
 0.37121006784339744,
 0.4887718670691053,
 0.601870933547616,
 0.6162089370191097,
 0.45837425521264474,
 0.3486258378252387,
 0.42296192737606664,
 0.5927434967597947,
 0.31822200395787753,
 0.1663989398938914,
 0.11217884440945151,
 0.049479120845596,
 0.021726015128660948,
 0.011777984634197007,
 0.008273614132000754,
 0.006156108344051366,
 0.004955486437150588,
 0.003969900178102156,
 0.0032945151052748162,
 0.0027849372709169984,
 0.00234560761794

In [ ]:
hist_val

[tensor(0.),
 tensor(0.),
 tensor(0.3333),
 tensor(0.),
 tensor(0.1667),
 tensor(1.),
 tensor(1.),
 tensor(0.3333),
 tensor(0.1667),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.6667),
 tensor(0.),
 tensor(0.1667),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.5000),
 tensor(0.5000),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.6667),
 tensor(0.5000),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.),
 tensor(0.5000),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.1667),
 tensor(0.1667),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.1667),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 tensor(0.3333),
 te

In [ ]:
hist_test_per_case = []
model.to(device)
case_test = []
for batch, data in enumerate(dataloader_test, 1):
            image = data['t1'].to(device)
            segment = data['label'].to(device)
            case_test.append(data['case'])
            with torch.set_grad_enabled(False):
                segment_hat = model(image)
                ac = accuracy(segment_hat.cpu(), segment.cpu())
            hist_test_per_case.append(ac)

In [ ]:
hist_test_per_case

NameError: name 'hist_test_per_case' is not defined

In [ ]:
case_test

[['105'], ['106'], ['108'], ['109'], ['110'], ['112'], ['113'], ['116']]

In [ ]:
hist_test

NameError: name 'hist_test' is not defined

In [ ]:
model_state

NameError: name 'model_state' is not defined

In [ ]:
hist_train_per_case = []
model.to(device)
case_train = []
for batch, data in enumerate(dataloader_train, 1):
            image = data['t1'].to(device)
            segment = data['label'].to(device)
            case_train.append(data['case'])
            with torch.set_grad_enabled(False):
                segment_hat = model(image)
                ac = accuracy(segment_hat.cpu(), segment.cpu())
            hist_train_per_case.append(ac)

In [ ]:
hist_train_per_case

[tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.)]

In [ ]:
case_train

[['18'],
 ['96'],
 ['70'],
 ['69'],
 ['16'],
 ['98'],
 ['54'],
 ['76'],
 ['12'],
 ['92'],
 ['73'],
 ['25'],
 ['28'],
 ['39'],
 ['95'],
 ['102'],
 ['72'],
 ['20'],
 ['100'],
 ['78'],
 ['94'],
 ['29'],
 ['26'],
 ['83'],
 ['24'],
 ['6'],
 ['101'],
 ['71'],
 ['4'],
 ['77']]